## <p style="text-align: center;">Numerical Algorithms - Homework Assignment 3</p>
**<p style="text-align: center;">VU Numerical Algorithms, summer semester 2018. Due to 17.06.2018. </p>**

### Programming Exercises

#### Effects of Preconditioners on Conjugate Gradient (12 points)

Implement the conjugate gradient method *CG* and experimentally investigate the effect of different preconditioners on its convergence. Please compare standard *CG* and preconditioned *CG* (*PCG*) for three given symmetric positive definite test problems in terms of the number of iterations and in terms of the runtime (including the time for
computing and applying the preconditioner!) until convergence. Show the convergence histories (norm of relative residual vs. iteration number) graphically.

* Implement standard CG efficiently (do not use the CG implementation available in Octave!). In particular, store the sparse matrix in a sparse matrix format
* Use the following preconditioners:
Diagonal preconditioner
  * Block diagonal preconditioner
  * Incomplete Cholesky factorization with no fill-in
  * Incomplete Cholesky factorization with threshold dropping: experiment with different thresholds and discuss the effect of the choice of threshold.
* Test matrices: Please use (at least) the following three test matrices from the
"Matrix Market" (http://math.nist.gov/MatrixMarket/) for your experiments:
  * http://math.nist.gov/MatrixMarket/data/Harwell-Boeing/lanpro/nos5.html
  * http://math.nist.gov/MatrixMarket/data/Harwell-Boeing/lanpro/nos6.html
  * http://math.nist.gov/MatrixMarket/data/misc/cylshell/s3rmt3m3.html

In [6]:
%plot -f svg

% Import routines needed to execute assignment.
source("source/assignment3.m")
% Import routines for reading matrix market (.mtx) format.
% Source: https://math.nist.gov/MatrixMarket/mmio/matlab/mmiomatlab.html.
source("source/mminfo.m")
source("source/mmread.m")

% Execute code for programming assignment.
function execute()
    % Iterate over test matrices.
    filenames = {"nos5.mtx"}%; "nos6.mtx"; "s3rmt3m3.mtx"};
    for i = 1:length(filenames)
        % Load file.
        [A, rows, cols, entries, rep, field, symm] = mmread(["data/" filenames{i}]);

        % Prepare data structure for evaluation metrics.
        res.st = struct();
        res.pc = struct();
        
        % ----------------------------------------------------
        % 1. Apply standard CG.
        % ----------------------------------------------------
        
        [
            res.st.numberOfIterations, ...
            res.st.runTime, ...
            res.st.relativeResidualHistory
        ] = applyCG(A, 10^-4);
        
        % ----------------------------------------------------
        % 2. Apply CG with preconditioner.
        % ----------------------------------------------------
        
        pcNames = {"diag"; "block"; "cholesky_nofi"; "cholesky_drop"};
        for i = 1:length(pcNames)
            pcNames{i}
            
            % Configure preconditioner.
            % Note: Parameters used here were determined in prior experiments (see documentation).
            precondConfig = struct();
            precondConfig.method = pcNames{i};
            precondConfig.alpha = 3;
            precondConfig.stepsize = 3;
            precondConfig.dropThreshold = 3;
            % Generate matrix for preconditioning.
            M = generatePreconditioning(A, precondConfig);
            
            % Apply CG with preconditioning.
            [
                res.pc.(pcNames{i}).numberOfIterations, ...
                res.pc.(pcNames{i}).runTime, ...
                res.pc.(pcNames{i}).relativeResidualHistory
            ] = applyCG(A, 10^-2, inv(M));
        end
        
        
    end
end

% Investigate effect of threshold for PC with 
% incomplete Cholesky factorization with threshold 
% dropping.
function executeParameterGridSearch()
    % 1. Test block size.
    
    % 2. 
end

% Execute parameter grid search to find decent values for
%    - alpha
%    - stepsize
%    - dropThreshold.
% Note that this search is variant towards the investigated PC-method.
executeParameterGridSearch()

% Execute task.
execute()

filenames = 
{
  [1,1] = nos5.mtx
}
ans = diag
ans = block
ans = cholesky_nofi
ans = cholesky_drop
